In [ ]:
import tensorflow as tf
from d2l import tensorflow as d2l

In [ ]:
def net():
    return tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=6, kernel_size=5, activation='sigmoid',
                               padding='same'),
        tf.keras.layers.AvgPool2D(pool_size=2, strides=2),
        tf.keras.layers.Conv2D(filters=16, kernel_size=5,
                               activation='sigmoid'),
        tf.keras.layers.AvgPool2D(pool_size=2, strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(120, activation='sigmoid'),
        tf.keras.layers.Dense(84, activation='sigmoid'),
        tf.keras.layers.Dense(10)])

        

In [ ]:
X = tf.random.uniform((1, 28, 28, 1))
for layer in net().layers:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape: \t', X.shape)

In [ ]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

In [ ]:
class TrainCallback(tf.keras.callbacks.Callback):  #@save
    """A callback to visiualize the training progress."""
    def __init__(self, net, train_iter, test_iter, num_epochs, device_name):
        self.timer = d2l.Timer()
        self.animator = d2l.Animator(
            xlabel='epoch', xlim=[1, num_epochs], legend=[
                'train loss', 'train acc', 'test acc'])
        self.net = net
        self.train_iter = train_iter
        self.test_iter = test_iter
        self.num_epochs = num_epochs
        self.device_name = device_name
    def on_epoch_begin(self, epoch, logs=None):
        self.timer.start()
    def on_epoch_end(self, epoch, logs):
        self.timer.stop()
        test_acc = self.net.evaluate(
            self.test_iter, verbose=0, return_dict=True)['accuracy']
        metrics = (logs['loss'], logs['accuracy'], test_acc)
        self.animator.add(epoch + 1, metrics)
        if epoch == self.num_epochs - 1:
            batch_size = next(iter(self.train_iter))[0].shape[0]
            num_examples = batch_size * tf.data.experimental.cardinality(
                self.train_iter).numpy()
            print(f'loss {metrics[0]:.3f}, train acc {metrics[1]:.3f}, '
                  f'test acc {metrics[2]:.3f}')
            print(f'{num_examples / self.timer.avg():.1f} examples/sec on '
                  f'{str(self.device_name)}')

#@save
def train_ch6(net_fn, train_iter, test_iter, num_epochs, lr, device):
    """Train a model with a GPU (defined in Chapter 6)."""
    device_name = device._device_name
    strategy = tf.distribute.OneDeviceStrategy(device_name)
    with strategy.scope():
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        net = net_fn()
        net.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    callback = TrainCallback(net, train_iter, test_iter, num_epochs,
                             device_name)
    net.fit(train_iter, epochs=num_epochs, verbose=0, callbacks=[callback])
    return net

In [ ]:
lr, num_epochs = 0.9, 10
train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())


Replace the average pooling with maximum pooling. What happens?

In [ ]:
def net():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters = 6, kernel_size = 5, activation=tf.nn.sigmoid, padding='same'), 
        tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = (2, 2)),
        tf.keras.layers.Conv2D(filters=16, kernel_size=5, activation=tf.nn.sigmoid),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(120, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(84, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(10)
    ])

In [ ]:
lr, num_epochs = 0.9, 10
train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

Try to construct a more complex network based on LeNet to improve its accuracy.



Adjust the convolution window size.

Adjust the activation function (e.g., ReLU).

Adjust the learning rates and other training details (e.g., initialization and number of epochs.)

In [ ]:
import keras_tuner as kt
from  keras_tuner import HyperParameters as hp

In [ ]:
def net(hp: kt.HyperParameters):
    activation = hp.Choice( name = 'activation', values = ['sigmoid', 'elu', 'relu'], ordered = False)
    filters = hp.Choice(name = 'filters', values = [4, 8, 16, 32])
    kernel_size = hp.Choice(name = 'kernel_size', values =[3, 5, 7])
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    optimzer = tf.keras.optimizers.SGD(learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    net = tf.keras.Sequential()
    for _ in range(hp.Int('conv_layers', 1, 3, default=3)):
            net.add(tf.keras.layers.Conv2D(filters , kernel_size, activation= activation))
    net.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)))           
    net.add(tf.keras.layers.Flatten())
    net.add(tf.keras.layers.Dense(hp_units, activation=tf.nn.sigmoid))
    net.add(tf.keras.layers.Dense(hp_units, activation=tf.nn.sigmoid))
    net.add(tf.keras.layers.Dense(10))
    net.compile(optimizer = optimzer, loss = loss, metrics = ['accuracy'])
    return net

In [ ]:
tuner = kt.Hyperband(
    net,
    objective=kt.Objective('accuracy', direction='max'),max_epochs=50, factor=2,directory="results_dir",
    project_name="mnist", distribution_strategy=tf.distribute.OneDeviceStrategy(d2l.try_gpu()._device_name), overwrite=True, seed=42)


In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(train_iter, epochs = 50, workers = 8, use_multiprocessing=True, callbacks = [tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)])

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
best_model =  tuner.get_best_models()[0]

In [ ]:

best_model.evaluate(test_iter)